In [1]:
from os.path import isfile, join, exists
from os import listdir, makedirs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from brainpipe.system import study

In [42]:
def def_freqname(freq):
    freqnames = ['0_theta', '1_alpha', '2_beta', '3_gamma']
    freqname = freqnames[freq]
    return freqname

def odor_su_dict():
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
    return odors_su

def norm_power_by_roi(phase,su,roi,freq,val):
    if val == 'xpow':
        path_pow = join(st.path, 'feature/0_Power_'+phase+'_By_Odor/')
        mat_file = path_pow+'{}_odor_{}_bipo_sel_physFT_pow.npz'
    elif val == 'tps':
        path_pow = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_all/TPS_R_p_by_odor/')
        mat_file = join(path_pow, 'TPS_spear_{}_odor_{}_{}.npz')
        
    all_odor_pow, scores = np.array([]), []
    odors_su = odor_su_dict()
    fname = def_freqname(freq)
    for odor in odors_su[su]:
        if val == 'tps':
            mat = np.load(mat_file.format(su,odor,fname))
            xpow, Mai_RL, channels = mat[val], mat['label'], mat['channel']
        else:
            mat = np.load(mat_file.format(su,odor))
            xpow, Mai_RL, channels = mat[val][freq,...], mat['Mai_RL'], mat['channels']      
        if roi == 'pPirT':
            id_rois = [r in ['pPirT','Amg','Amg-PirT'] for r in Mai_RL]
        else:
            id_rois = [r == roi for r in Mai_RL]
        xpow = np.mean(xpow[id_rois,27:47,:],axis=-2) if val == 'xpow' else xpow[id_rois,...]
        score_roi = [odors_su[su][odor]]*xpow.shape[-1]
        scores.append(score_roi)
        all_odor_pow = np.concatenate((all_odor_pow,xpow),axis=-1) if np.size(all_odor_pow) else xpow
    scores = np.concatenate(scores,axis=0)
    kwargs = {}
    kwargs['label'], kwargs['channels']= Mai_RL[id_rois], channels[id_rois]
    kwargs['xyz'], kwargs[val] = mat['xyz'][id_rois], all_odor_pow
    kwargs['fname'], kwargs['nelecs'] = fname, all_odor_pow.shape[0]
    kwargs['scores'] = np.array(scores)
    return kwargs

def groups_pow_scores(phase,su,roi,freq,val):
    dict_ = norm_power_by_roi(phase,su,roi,freq,val)
    scores = dict_['scores']
    pow_norm = dict_[val]
    #print(len(scores),pow_norm.shape)
#     steps = np.percentile(scores,[0,33,66,100])
    steps = np.percentile(scores,[0,25,50,75,100])
    pow_groups = []
    if pow_norm.shape[0] == 1:
        for g,step in enumerate(steps[:-1]):
            pow_groups.append(pow_norm[:,np.where((step<=scores)&(scores<=(steps[g+1])))[0]])
    else:
        for g,step in enumerate(steps[:-1]):
            pow_groups.append(np.squeeze(pow_norm[:,np.where((step<=scores)&(scores<=(steps[g+1])))]))
            
    print('groups shapes',[pow.shape for pow in pow_groups])
    labels, weights = [], []
    for i,_ in enumerate(pow_groups):
        labels.append([i]*pow_groups[i].shape[1])
        weights.append([pow_groups[i].shape[1]])
    labels = np.concatenate(labels,axis=0)
    weights = np.concatenate(weights,axis=0)
    return pow_groups, labels, weights, dict_

In [43]:
from joblib import Parallel, delayed
from scipy import stats

st = study('Olfacto')
save_path = join(st.path, 'kruskal_pow_all_trials/{}/all_elecs_by_subj_sig{}/')
f_form_save = join(save_path,'{}/Kruskal_{}_{}_{}.png')
subjects = ['LEFC','VACJ','SEMC','FERJ','PIRJ','CHAF']
rois = ['ACC','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
phase = 'Encoding'
groups = 4
val = 'xpow'

def kruskal_plot_by_subj(su,plot=False):
    pvals, K_vals = [], []
    for freq in range(4):
        for roi in rois:
            pow_groups, labels, weights, dict_ = groups_pow_scores(phase,su,roi,freq,val)
            freq_name, nelecs = dict_['fname'], dict_['nelecs']
            if nelecs > 0:
                for elec_num in range(nelecs): 
                    elec, elec_label = dict_['channels'][elec_num], dict_['label'][elec_num]
                    xyz = dict_['xyz'][elec_num]
                    pow_data_elec = []
                    for i,power in enumerate(pow_groups):
                        pow_data_elec.append(power[elec_num])
                        
                    #Compute the statistics (Kruskal-Wallis)
                    K,p = stats.kruskal(pow_data_elec[0],pow_data_elec[1],
                                        pow_data_elec[2],pow_data_elec[3])
                    print('stats for',freq,roi,K,p)
                    pvals = np.append(pvals,p)
                    inc = all(np.mean(i) < np.mean(j) for i, j in zip(pow_data_elec, pow_data_elec[1:])) 
                    dec = all(np.mean(i) > np.mean(j) for i, j in zip(pow_data_elec, pow_data_elec[1:]))
                    diff = True if inc == True or dec == True else False
                    if plot == True and p <= 0.05 and diff == True:
                        fig, ax = plt.subplots()
                        plt.title('Perf & '+freq_name[2:]+' power in '+elec+' ('+roi+') for '+su)
                        plt.xlabel('EpiScore'), plt.ylabel(freq_name[2:])
                        anchored_text = AnchoredText('K = %s, p = %s' % (K, p), loc=2)
                        ax.add_artist(anchored_text)
                        xticks, w = np.arange(groups), 0.8
                        means = [pow.mean()for pow in pow_data_elec]
                        stds = np.array([stats.sem(pow) for pow in pow_data_elec])
                        plt.bar(xticks,means,color='blue',yerr=stds)
                        path_plot = save_path.format(phase,groups) + freq_name+'/'
                        if not exists(path_plot):
                            makedirs(path_plot)
                        plt.savefig(path_plot+'{}_{}_Kruskal_{}_{}_{}.png'.format(roi,freq_name[2:],su,elec,'EpiScore'))
                        plt.clf()
                        plt.close()
    np.save(save_path.format(phase,groups)+'{}_pvals_Krusk_{}groups.npy'.format(su,groups),pvals)

def fdr_pvals(su,phase,groups,alpha):
    from mne.stats import bonferroni_correction, fdr_correction
    pvals = np.load(save_path.format(phase,groups)+su+'_pvals_Krusk_'+str(groups)+'groups.npy')
    pvals = pvals[np.logical_not(np.isnan(pvals))]
    _, pval_bonf = fdr_correction(pvals, alpha=alpha)
    print(pval_bonf[pval_bonf<0.05])
    return np.round(pval_bonf,3)
    
if __name__ == "__main__":
    Parallel(n_jobs=-1)(delayed(kruskal_plot_by_subj)(su,plot=True) for su in subjects)
#     kruskal_plot_by_subj('CHAF',plot=True)
    

-> Olfacto loaded


In [ ]:
from mne.stats import bonferroni_correction, fdr_correction
from scipy import stats

feat, corr, phase = 'xpow', False, 'Encoding'
groups = 4
###############################################################################
st = study('Olfacto')
if feat == 'tps':
    path_npz = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor/TPS_R_p_by_odor/')
    path2save = join(st.path, 'kruskal_TPSim_all_trials/'+phase+'/all_elecs_by_subj_sig'+str(groups)+'/')
    npz_form = join(path_npz, 'TPS_spear_{}_odor_{}.npz')
    lab_code = 'label'
elif feat == 'xpow':
    path_npz = join(st.path, 'feature/0_Power_'+phase+'_By_Odor/')
    path2save = join(st.path, 'kruskal_pow_all_trials/'+phase+'/all_elecs_by_subj_sig'+str(groups)+'/')
    npz_form = join(path_npz, '{}_odor_{}_bipo_sel_physFT_pow.npz')
    lab_code = 'Mai_RL'
f_form_save = join(path2save, 'Kruskal_{}_{}_{}.png')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
method = ['s_Mail_RL']
alpha = 0.05

rois_to_keep = ['ACC','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
'Amg','Amg-PirT'

for s,su in enumerate(odors_su):
    pvals = []
    for i,freq in enumerate(freqs):
        for roi in rois_to_keep:
            for odor in odors_su[su]:
                arch_sig = np.load(npz_form.format(su,odor))
                Mai_RL= arch_sig[lab_code]
                Mai_RL = ['pPirT' if r in ['Amg','Amg-PirT'] else r for r in Mai_RL]
                id_rois = [1 if r else 0 in rois_to_keep for r in Mai_RL]
                if feat == 'xpow':
                    power = np.mean(arch_sig[feat][i,:,27:47,:],axis=(1))
                elif feat == 'tps':
                    power = arch_sig[feat]
                power = power[id_rois]
            
            power_roi = np.ndarray.flatten(power_roi)
            score_roi = [odors_su[su][odor]]*power_roi.shape[0]
            pow_su = np.append(pow_su,power_roi)
            score_su = np.append(score_su, score_roi)
            pow_su_norm = (pow_su - pow_su.mean())/pow_su.std()
            score_su_norm = (score_su - score_su.mean())/score_su.std()
            pow_odors = np.append(pow_odors,pow_su_norm)
            scores_odors = np.append(scores_odors,score_su_norm)
            pow_raw = np.append(pow_raw,pow_su)

        #Split the data into 3 groups
        print('before concat',scores_odors.shape, pow_odors.shape)
        steps = np.percentile(scores_odors,[0,20,40,60,80,100])
        pow_groups, scores_groups, pow_raw_groups = [], [], []
        for g in range(groups):
            pow_groups.append(pow_odors[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
            pow_raw_groups.append(pow_raw[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
        print([pow.shape for pow in pow_groups], [pow.shape for pow in pow_raw_groups])
        
        #Compute the statistics (Kruskal-Wallis)
        K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2])
        print('stats for',freq,region,K,p)
        pvals = np.append(pvals,p)
        #if p < 0.01:
        fig, ax = plt.subplots()
        plt.title('Epi score and '+freq[2:]+' power in '+region)
        plt.xlabel('EpiScore'), plt.ylabel(freq)
        anchored_text = AnchoredText('K = %s, p = %s' % (K, p), loc=2)
        ax.add_artist(anchored_text)
        xticks, w = np.arange(0,groups), 0.8
        means = [pow.mean()for pow in pow_groups]
        stds = [stats.sem(pow) for pow in pow_groups]
        plt.bar(xticks,means,color='blue',yerr=stds)
        plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score'))
        plt.clf()
        plt.close()
pvals = np.array(pvals)[np.logical_not(np.isnan(pvals))]
## Determine the threshold p value for FDR correction
_, pval_bonf = bonferroni_correction(pvals, alpha=alpha)
print(np.round(pval_bonf,3))

_, pval_fdr = fdr_correction(pvals, alpha=alpha, method='indep')
print(np.round(pval_fdr,3))
        

### Plot correlation btw power and score in signif elecs // normalized by subject

In [ ]:
from mne.stats import bonferroni_correction, fdr_correction
from scipy import stats

feat, corr, phase, win,th, alpha = 'pow', False, 'Encoding', 1.0,'0.01',0.05
groups = 4
###############################################################################
st = study('Olfacto')
###############################################################################
path_npz = join(st.path, 'feature/0_Power_'+phase+'_By_Odor/')
npz_form = join(path_npz, '{}_odor_{}_bipo_sel_physFT_pow.npz')
path_classif = join(st.path, 
   'figure/0_Classif_Power_'+phase[0]+'_EpiPerf_LowHigh_1000perm_BBG/')
classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_form = join(path_classif, 
   'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
path2save = join(st.path, 
   'kruskal_pow_all_trials/'+phase+'/'+phase.lower()+'_signif_elecs'+str(groups)+'/')
f_form_save = join(path2save, 'Kruskal_{}_{}_{}_{}.pdf')
###############################################################################
# path_classif = join(st.path, 'figure/Classif_TPSim_Encoding_Retrieval_Odor_1000perm/')
# classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
# masks_form = join(path_classif, 'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
# path_npz = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor/TPS_R_p_by_odor/')
# path2save = join(st.path, 'kruskal_TPSim_all_trials/'+phase+'/signif_elecs_'+str(groups)+'/')
# npz_form = join(path_npz, 'TPS_spear_{}_odor_{}.npz')
# f_form_save = join(path2save, 'Kruskal_{}_{}_{}.pdf')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

su_codes = [['S0','S1','S2','S3','S4','S5'] if phase == 'Encoding' 
            else ['S0','S1','S2','S3','S4','S5']]
#freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
method = ['s_Mail_RL']

if phase == 'Encoding':
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'MICP': {2:6,12:8,6:96,8:8,3:12,18:4,9:6,14:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
else:
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68},#15:4 #remove for TPSim 15:4
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

rois_list = {'Frontal':['ACC','IFG','MFG','SFG'],
            'Olf':['pPirT','Amg-PirT','OFC'],
            'MTL':['HC']}
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']

pvals, H_vals = [], []
for i,freq in enumerate(freqs): #(2,7)
    for region in sorted(rois_list):
        mat = np.load(classif_npz.format('All_subjects',freq, 'odor'))
        id_rois = np.where([roi in rois_to_keep for roi in mat['s_labels']])
        subjects = mat['su_codes'][id_rois]
        mask = np.load(masks_form.format(freq,str(win),th))
        mask = np.logical_not(mask) #inverse of visbrain 
        pow_odors, pow_raw,scores_odors, su_list = [], [], [],[]
        for s,su in enumerate(sorted(odors_su)):
            mask_su = mask[np.where(subjects=='S'+str(s))]
            #print(su,'mask su',mask_su.shape)
            pow_su, score_su = [], []
            for roi in rois_list[region]:
                for odor in odors_su[su]:
                    arch_sig = np.load(npz_form.format(su,odor))
                    id_roi_su = np.where([roi in rois_to_keep for roi in arch_sig['Mai_RL']])
                    Mai_RL= arch_sig['Mai_RL'][id_roi_su][mask_su] #'Mai_RL'
                    #print(Mai_RL.shape, arch_sig['xpow'].shape)
                    power = np.mean(arch_sig['xpow'][i,:,27:47,:],axis=(1))[id_roi_su][mask_su] #nfreq,nelecs,npts,ntrials
                    #power = arch_sig['xpow'][i,:,20,:][mask_su] #rest
                    #power = arch_sig['tps'][i,:,:][mask_su] 
                    power_roi = power[np.where(Mai_RL==roi)]
                    power_roi = np.ndarray.flatten(power_roi)
                    score_roi = [odors_su[su][odor]]*power_roi.shape[0]
                    pow_su = np.append(pow_su,power_roi)
                    score_su = np.append(score_su,score_roi)
            pow_su_norm = (pow_su - pow_su.mean())/pow_su.std()
            score_su_norm = (score_su - score_su.mean())/score_su.std()
            pow_odors = np.append(pow_odors,pow_su_norm)
            scores_odors = np.append(scores_odors,score_su_norm)
            pow_raw = np.append(pow_raw,pow_su)
       #Split the data into 3 groups
        #print('before concat',scores_odors.shape, pow_odors.shape)
        if scores_odors.shape[0] != 0:
            if groups == 5:
                steps = np.percentile(scores_odors,[0,20,40,60,80,100])
            elif groups == 4:
                steps = np.percentile(scores_odors,[0,25,50,75,100])
            elif groups ==3:
                steps = np.percentile(scores_odors,[0,33,66,100])
            #print(scores_odors,steps)
            pow_groups, scores_groups, pow_raw_groups = [], [], []
            for g in range(groups):
                pow_groups.append(pow_odors[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
                pow_raw_groups.append(pow_raw[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
            print([pow.shape for pow in pow_groups], [pow.shape for pow in pow_raw_groups])

            #Compute the statistics (Kruskal-Wallis)
            if groups == 5:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                                pow_groups[3],pow_groups[4]) #pow_groups[4]
            elif groups == 4:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                             pow_groups[3]) #pow_groups[4]
            elif groups == 3:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2])
            print('stats for',freq,region,K,p)
            pvals = np.append(pvals,p)
            H_vals = np.append(H_vals, K)
            #if p < 0.01:
            fig, ax = plt.subplots()
            plt.title('Epi score and '+freq[2:]+' power in '+region)
            plt.xlabel('EpiScore'), plt.ylabel(freq)
            anchored_text = AnchoredText('K = %s, p = %s' % (K, p), loc=2)
            ax.add_artist(anchored_text)
            xticks, w = np.arange(0,groups), 0.8
            means = [pow.mean()for pow in pow_groups]
            stds = [stats.sem(pow) for pow in pow_groups]
            plt.bar(xticks,means,color='blue',yerr=stds)
            plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score'))
            plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score').replace('pdf','png'))
            plt.clf()
            plt.close()
            
pvals = np.array(pvals)[np.logical_not(np.isnan(pvals))]
## Determine the threshold p value for FDR correction
_, pval_bonf = bonferroni_correction(pvals, alpha=alpha)
print(np.round(pval_bonf,3))

_, pval_fdr = fdr_correction(pvals, alpha=alpha, method='indep')
print(np.round(pval_fdr,3))
print('H values', np.round(H_vals,3))


In [ ]:
from mne.stats import bonferroni_correction, fdr_correction
from scipy import stats

feat, corr, phase, win,th, alpha = 'tps', False, 'Retrieval', 1.0,'0.01',0.05
groups = 4
###############################################################################
st = study('Olfacto')
###############################################################################
# path_npz = join(st.path, 'feature/0_Power_'+phase+'_By_Odor/')
# npz_form = join(path_npz, '{}_odor_{}_bipo_sel_physFT_pow.npz')
# path_classif = join(st.path, 
#    'figure/0_Classif_Power_'+phase[0]+'_EpiPerf_LowHigh_1000perm_BBG/')
# classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
# masks_form = join(path_classif, 
#    'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
# path2save = join(st.path, 
#    'kruskal_pow_all_trials/'+phase+'/'+phase.lower()+'_signif_elecs'+str(groups)+'/')
# f_form_save = join(path2save, 'Kruskal_{}_{}_{}_{}.pdf')
###############################################################################
path_classif = join(st.path, 'figure/LDA_TPSim_E_R_all_BBG/')
classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_form = join(path_classif, 'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
path_npz = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_all/TPS_R_p_by_odor/')
#path2save = join(st.path, 'kruskal_TPSim_all_trials/TPS_signif_elecs_signif'+str(groups)+'/')
path2save = join(st.path, 'kruskal_TPSim_all_trials/Correl_TPSim_signif'+str(groups)+'/')
npz_form = join(path_npz, 'TPS_spear_{}_odor_{}_{}.npz')
f_form_save = join(path2save, 'Correl_{}_{}_{}.pdf')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

su_codes = [['S0','S1','S2','S3','S4','S5','S6'] if phase == 'Encoding' 
            else ['S0','S1','S2','S3','S4','S5']]
#freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
method = ['s_Mail_RL']

if phase == 'Encoding':
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'MICP': {2:6,12:8,6:96,8:8,3:12,18:4,9:6,14:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
else:
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68},#15:4 #remove for TPSim 15:4
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']
rois = ['ACC','Amg','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']

pvals, H_vals, r_vals = [], [], []
for i,freq in enumerate(freqs):
    #Load all data classif 
    mat = np.load(classif_npz.format('All_subjects',freq, 'odor'))
    #Filter matrix for MTl, Olf and Frontal regions
    id_rois = np.where([roi in rois_to_keep for roi in mat['s_labels']])
    subjects = mat['su_codes'][id_rois]
    #Load mask for significant elecs
    mask = np.load(masks_form.format(freq,str(win),th))
    mask = np.logical_not(mask) #transform inverse of visbrain 
    
    #Correlation by freq and by region for all subjects
    for region in sorted(rois):
        pow_odors,pow_raw, scores_odors, su_list = [], [], [],[]
        for s,su in enumerate(sorted(odors_su)):
            mask_su = mask[np.where(subjects=='S'+str(s))]
            pow_su, score_su = [], []
            for odor in odors_su[su]:
                arch_sig = np.load(npz_form.format(su,odor,freq))
                #Take only SIGNIF electrodes in MTL, Olf, Frontal
                id_roi_su = np.where([roi in rois_to_keep for roi in arch_sig['label']])
                Mai_RL= arch_sig['label'][id_roi_su][mask_su]
#                 power = np.mean(arch_sig['xpow'][i,:,27:47,:],axis=(1))[id_roi_su][mask_su] #nfreq,nelecs,npts,ntrials
                power = arch_sig['tps'][id_roi_su][mask_su] #nfreq,nelecs,npts,ntrials
                if region == 'pPirT':
                    power_roi = power[np.where(roi in ['pPirT','Amg-PirT'] for roi in Mai_RL)]
                else:
                    power_roi = power[np.where(Mai_RL==region)]
                power_roi = np.ndarray.flatten(power_roi)
                score_roi = [odors_su[su][odor]]*power_roi.shape[0]
                pow_su = np.append(pow_su,power_roi)
                score_su = np.append(score_su,score_roi)
            pow_su_norm = (pow_su - pow_su.mean())/pow_su.std()
            score_su_norm = (score_su - score_su.mean())/score_su.std()
            pow_odors = np.append(pow_odors,pow_su_norm)
            scores_odors = np.append(scores_odors,score_su_norm)
            pow_raw = np.append(pow_raw,pow_su)
       #Split the data into 3 groups
        #print('before concat',scores_odors.shape, pow_odors.shape)
        if scores_odors.shape[0] != 0:
            if groups == 5:
                steps = np.percentile(scores_odors,[0,20,40,60,80,100])
            elif groups == 4:
                steps = np.percentile(scores_odors,[0,25,50,75,100])
            elif groups ==10:
                steps = np.percentile(scores_odors,[0,10,20,30,40,50,60,70,80,90,100])
            elif groups ==3:
                steps = np.percentile(scores_odors,[0,33,66,100])
            #print(scores_odors,steps)
            pow_groups, scores_groups, pow_raw_groups = [], [], []
            for g in range(groups):
                pow_groups.append(pow_odors[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
                pow_raw_groups.append(pow_raw[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
            print([pow.shape for pow in pow_groups], [pow.shape for pow in pow_raw_groups])

            #Compute the statistics (Kruskal-Wallis)
            if groups == 5:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                                pow_groups[3],pow_groups[4]) #pow_groups[4]
            if groups == 3:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2])
            elif groups == 4:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                             pow_groups[3]) #pow_groups[4]
            elif groups == 10:
                K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],pow_groups[3],
                        pow_groups[4],pow_groups[5],pow_groups[6],pow_groups[7],
                        pow_groups[8],pow_groups[9])
            print('stats for',freq,region,K,p)
            pvals = np.append(pvals, p)
            H_vals = np.append(H_vals, K)
            r_vals = np.append(r_vals, region)
            #if p < 0.05:
            fig, ax = plt.subplots()
            plt.title('Epi score and '+freq[2:]+' power in '+region)
            plt.xlabel('EpiScore'), plt.ylabel(freq)
            anchored_text = AnchoredText('K = %s, p = %s' % (K, p), loc=2)
            ax.add_artist(anchored_text)
            xticks, w = np.arange(0,groups), 0.8
            means = [pow.mean()for pow in pow_groups]
            stds = [stats.sem(pow) for pow in pow_groups]
            plt.bar(xticks,means,color='blue',yerr=stds)
            plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score'))
            plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score').replace('pdf','png'))
            plt.clf()
            plt.close()
print(r_vals)
pvals = np.array(pvals)[r_vals=='HC']
H_vals = np.array(H_vals)[r_vals=='HC']
#pvals = np.array(pvals)[np.logical_not(np.isnan(pvals))]
## Determine the threshold p value for FDR correction
_, pval_bonf = bonferroni_correction(pvals, alpha=alpha)
#print(np.round(pval_bonf,3))

_, pval_fdr = fdr_correction(pvals, alpha=alpha, method='indep')
print(np.round(pval_fdr,3))
print(pval_fdr)
print('H values', np.round(H_vals,2))

In [ ]:
from mne.stats import bonferroni_correction, fdr_correction
from scipy import stats

feat, corr, phase, win,th, alpha = 'tps', False, 'Encoding', 1.0,'0.01',0.05
groups = 4
###############################################################################
st = study('Olfacto')
###############################################################################
path_npz = join(st.path, 'feature/0_Power_'+phase+'_By_Odor/')
npz_form = join(path_npz, '{}_odor_{}_bipo_sel_physFT_pow.npz')
path_classif = join(st.path, 
   'figure/0_Classif_Power_'+phase[0]+'_EpiPerf_LowHigh_1000perm_BBG/')
classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_form = join(path_classif, 
   'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
path2save = join(st.path, 
   'kruskal_pow_all_trials/'+phase+'/by_subj_signif'+str(groups)+'/')
f_form_save = join(path2save, 'Kruskal_{}_{}_{}_{}_{}.pdf')
###############################################################################
#path_classif = join(st.path, 'figure/LDA_TPSim_E_R_all_BBG/')
#classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
#masks_form = join(path_classif, 'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
#path_npz = join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_all/TPS_R_p_by_odor/')
#path2save = join(st.path, 'kruskal_TPSim_all_trials/TPS_signif_elecs_signif'+str(groups)+'/')
#path2save = join(st.path, 'kruskal_TPSim_all_trials/by_subj_signif'+str(groups)+'/')
#npz_form = join(path_npz, 'TPS_spear_{}_odor_{}_{}.npz')
#f_form_save = join(path2save, 'Correl_{}_{}_{}.pdf')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

su_codes = [['S0','S1','S2','S3','S4','S5','S6'] if phase == 'Encoding' 
            else ['S0','S1','S2','S3','S4','S5']]
#freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']
method = ['s_Mail_RL']

if phase == 'Encoding':
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'MICP': {2:6,12:8,6:96,8:8,3:12,18:4,9:6,14:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
else:
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68},#15:4 #remove for TPSim 15:4
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
rois_to_keep = ['ACC','Amg','Amg-PirT','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']
rois = ['ACC','HC','IFG','Ins','MFG','OFC','PHG',
                'SFG','pPirT']

for i,freq in enumerate(freqs):
    #Load all data classif 
    mat = np.load(classif_npz.format('All_subjects',freq, 'odor'))
    #Filter matrix for MTl, Olf and Frontal regions
    id_rois = np.where([roi in rois_to_keep for roi in mat['s_labels']])
    subjects = mat['su_codes'][id_rois]
    #Load mask for significant elecs
    mask = np.load(masks_form.format(freq,str(win),th))
    mask = np.logical_not(mask) #transform inverse of visbrain 
    
    #Correlation by freq and by region for all subjects
    for region in sorted(rois):
        pow_su_norm, score_su_norm = [], []
        for s,su in enumerate(sorted(odors_su)):
            mask_su = mask[np.where(subjects=='S'+str(s))]
            pow_su, score_su = [], []
            for odor in odors_su[su]:
                arch_sig = np.load(npz_form.format(su,odor,freq))
                #print(arch_sig.files)
                #Take only SIGNIF electrodes in MTL, Olf, Frontal
                id_roi_su = np.where([roi in rois_to_keep for roi in arch_sig['Mai_RL']])
                Mai_RL= arch_sig['Mai_RL'][id_roi_su][mask_su]
                RL = arch_sig['xyz'][id_roi_su][mask_su]
                RL = ['R' if x[0] > 0 else 'L' for x in RL ]
                power = np.mean(arch_sig['xpow'][i,:,27:47,:],axis=(1))[id_roi_su][mask_su] #nfreq,nelecs,npts,ntrials
                #power = arch_sig['tps'][id_roi_su][mask_su] #nfreq,nelecs,npts,ntrials
                if region == 'pPirT':
                    power_roi = power[[roi in ['pPirT','Amg','Amg-PirT'] for roi in Mai_RL]]
                else:
                    power_roi = power[np.where(Mai_RL==region)]
                powR, powL = [], []
                for i,p in enumerate(power_roi):
                    powR.append(p) if RL[i] == 'R' else powL.append(p)
                
                if np.size(np.array(powR)):
                    powR = np.array(powR)
                    score_roi = [odors_su[su][odor]]*powR.shape[0]
                    pow_su = np.append(pow_su,powR)
                    score_su = np.append(score_su,score_roi)
            pow_su_norm = (pow_su - pow_su.mean())/pow_su.std()
            score_su_norm = (score_su - score_su.mean())/score_su.std()
            if score_su_norm.shape[0] != 0:
                if groups == 5:
                    steps = np.percentile(score_su_norm,[0,20,40,60,80,100])
                elif groups == 4:
                    steps = np.percentile(score_su_norm,[0,25,50,75,100])
                elif groups ==10:
                    steps = np.percentile(score_su_norm,[0,10,20,30,40,50,60,70,80,90,100])
                elif groups ==3:
                    steps = np.percentile(score_su_norm,[0,33,66,100])
                #print(scores_odors,steps)
                pow_groups, scores_groups, pow_raw_groups = [], [], []
                for g in range(groups):
                    pow_groups.append(pow_su_norm[np.where((steps[g]<=score_su_norm)&(score_su_norm<=(steps[g+1])))])
                print([pow.shape for pow in pow_groups])

                #Compute the statistics (Kruskal-Wallis)
                if groups == 5:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                                    pow_groups[3],pow_groups[4]) #pow_groups[4]
                if groups == 3:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2])
                elif groups == 4:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                                 pow_groups[3]) #pow_groups[4]
                elif groups == 10:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],pow_groups[3],
                            pow_groups[4],pow_groups[5],pow_groups[6],pow_groups[7],
                            pow_groups[8],pow_groups[9])
                print('stats for',freq,su,region,K,p)
                pvals = np.append(pvals, p)
                H_vals = np.append(H_vals, K)
                r_vals = np.append(r_vals, region)
                fig, ax = plt.subplots()
                plt.title(su+' // Epi score and '+freq[2:]+' power in '+region)
                plt.xlabel('EpiScore //'+str(RL)), plt.ylabel(freq)
                anchored_text = AnchoredText('K = %s, p = %s' % (K, p), loc=2)
                ax.add_artist(anchored_text)
                xticks, w = np.arange(0,groups), 0.8
                means = [pow.mean()for pow in pow_groups]
                stds = [stats.sem(pow) for pow in pow_groups]
                plt.bar(xticks,means,color='blue',yerr=stds)
                plt.savefig(f_form_save.format(freq[2:],region,su,'EpiScore','z-score'))
                plt.savefig(f_form_save.format(freq[2:],region,su,'EpiScore','z-score').replace('pdf','png'))
                plt.clf()
                plt.close()

In [ ]:
freqnames = ['theta','alpha','beta','gamma1','gamma2']
_, pval_fdr = fdr_correction(pvals, alpha=0.05, method='indep')
pval_fdr = np.split(pval_fdr,[i for i in range(2,10,2)])
for f in range(len(pval_fdr)):
    pvalfdr = pval_fdr[f]
    for i,s in enumerate(pvalfdr):
        if s < 0.05:
            print('FDR','freq',freqnames[f],rois[i],'p',round(s,5))


### TR and memory scores using Kruskal

In [ ]:
from mne.stats import bonferroni_correction, fdr_correction
from scipy import stats

score = 'Rec'
feat, corr, phase, win,th, alpha = 'pow', False, 'Encoding', 1.0,'0.01',0.05
groups = 4
###############################################################################
st = study('Olfacto')
###############################################################################
path_npz = join(st.path, 'feature/0_Power_'+phase+'_EpiPerf_By_Odor/')
npz_form = join(path_npz, '{}_odor_{}_bipo_sel_physFT_pow.npz')
path_classif = join(st.path, 
   'figure/0_Classif_Power_'+phase[0]+'_EpiPerf_LowHigh_1000perm_v2/')
classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_form = join(path_classif, 
   'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
path2save = join(st.path, 
   'kruskal_pow_all_trials/'+phase+'/'+phase.lower()+'_signif_elecs'+str(groups)+'/')
f_form_save = join(path2save, 'TR_Kruskal_{}_{}_{}_{}.pdf')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method = ['s_Mail_RL']
###############################################################################

if score == 'Rec':
    odors_su = {'CHAF': {5:5.84,7:4.67,8:4.45,9:5.94,1:3.21,2:7.56,3:4.98,4:4.34},
            'LEFC': {1:5.86,3:6.05,4:4.74,14:4.03,15:4.16,16:4.01,17:5.52},
            'PIRJ': {4:5.71,9:4.2,1:3.67,18:6.26,6:4.79,5:6.04,7:2.28}, #missing odor 15 
            'MICP':{},
            'VACJ': {14:9.16,15:3.64,16:5.86,17:8.54,10:7.37,11:10.92,12:10.66,13:4.56},
            'SEMC': {10:7.57,11:11.32,12:8.14,13:4.17,5:9.10,7:5.05,8:9.11,9:6.62},
            'FERJ': {16:5.80,17:4.02,5:3.08,7:2.80,12:2.60,13:3.56,2:3.38,1:2.28}}

if score == 'Epi':
    odors_su = {'CHAF': {5:7.96,7:5.82,8:6.08,9:7.77,1:4.39,2:9.27,3:6.42,4:6.26},
            'LEFC': {1:6.82,3:8.02,4:8.41,14:5.35,15:12.41,16:4.92,17:6.79},
            'PIRJ': {4:9.33,9:7.54,1:4.81,18:8.11,6:10.57,5:9.83,7:4.98}, #missing odor 15 
            'MICP':{},
            'VACJ': {14:16.55,15:6.10,16:8.25,17:15.96,10:14.97,11:14.79,12:25.83,13:10.68},
            'SEMC': {10:8.51,11:13.00,12:9.93,13:5.48,5:10.05,7:6.35,8:10.26,9:7.83},
            'FERJ': {16:7.96,17:6.30,5:4.84,7:5.23,12:4.33,13:7.07,2:5.89,1:4.42}}

u_color = ["darkblue", "royalblue", "deepskyblue", "mediumspringgreen", "yellow", "darkorange","red"]
rois_to_keep = ['ACC','Amg','HC','IFG','Ins','MFG','OFC','PHG','SFG']

pvals, H_vals = [], []
for i,freq in zip(range(2,7),freqs):
    #Load all data classif 
    mat = np.load(classif_npz.format('All_subjects',freq, 'odor'))
    #Filter matrix for MTl, Olf and Frontal regions
    id_rois = np.where([roi in rois_to_keep for roi in mat['s_MAI_RL']])
    subjects = mat['su_codes'][id_rois]
    #Load mask for significant elecs
    mask = np.load(masks_form.format(freq,str(win),th))
    mask = np.logical_not(mask) #transform inverse of visbrain 
    
    #Correlation by freq and by region for all subjects
    for region in sorted(rois_to_keep):
        pow_odors, scores_odors, su_list = [], [], []
        for s,su in enumerate(sorted(odors_su)):
            if su =='MICP': 
                print('no RTs pass',su)
            else:
                mask_su = mask[np.where(subjects=='S'+str(s))]
                pow_su, score_su = [], []
                for odor in odors_su[su]:
                    arch_sig = np.load(npz_form.format(su,odor))
                    #Take only SIGNIF electrodes in MTL, Olf, Frontal
                    id_roi_su = np.where([roi in rois_to_keep for roi in arch_sig['Mai_RL']])
                    Mai_RL= arch_sig['Mai_RL'][id_roi_su][mask_su]
                    power = np.mean(arch_sig['xpow'][i,:,27:47,:],axis=(1))[id_roi_su][mask_su] #nfreq,nelecs,npts,ntrials
                    if region == 'Amg':
                        power_roi = power[np.where(roi in ['Amg','pPirT','Amg-PirT'] for roi in Mai_RL)]
                    else:
                        power_roi = power[np.where(Mai_RL==region)]
                    power_roi = np.ndarray.flatten(power_roi)
                    score_roi = [odors_su[su][odor]]*power_roi.shape[0]
                    pow_su = np.append(pow_su,power_roi)
                    score_su = np.append(score_su,score_roi)
                pow_su_norm = (pow_su - pow_su.mean())/pow_su.std()
                score_su_norm = (score_su - score_su.mean())/score_su.std()
                pow_odors = np.append(pow_odors,pow_su_norm)
                scores_odors = np.append(scores_odors,score_su_norm)
                pow_raw = np.append(pow_raw,pow_su)
           #Split the data into 3 groups
            #print('before concat',scores_odors.shape, pow_odors.shape)
            if scores_odors.shape[0] != 0:
                if groups == 5:
                    steps = np.percentile(scores_odors,[0,20,40,60,80,100])
                elif groups == 4:
                    steps = np.percentile(scores_odors,[0,25,50,75,100])
                elif groups ==3:
                    steps = np.percentile(scores_odors,[0,33,66,100])
                #print(scores_odors,steps)
                pow_groups, scores_groups, pow_raw_groups = [], [], []
                for g in range(groups):
                    pow_groups.append(pow_odors[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
                    pow_raw_groups.append(pow_raw[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
                print([pow.shape for pow in pow_groups], [pow.shape for pow in pow_raw_groups])

                #Compute the statistics (Kruskal-Wallis)
                if groups == 5:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                                    pow_groups[3],pow_groups[4]) #pow_groups[4]
                elif groups == 4:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2],
                                 pow_groups[3]) #pow_groups[4]
                elif groups == 3:
                    K,p = stats.kruskal(pow_groups[0],pow_groups[1],pow_groups[2])
                print('stats for',freq,region,K,p)
                pvals = np.append(pvals,p)
                H_vals = np.append(H_vals, K)
                #if p < 0.01:
                fig, ax = plt.subplots()
                plt.title('Epi score and '+freq[2:]+' power in '+region)
                plt.xlabel('EpiScore'), plt.ylabel(freq)
                anchored_text = AnchoredText('K = %s, p = %s' % (K, p), loc=2)
                ax.add_artist(anchored_text)
                xticks, w = np.arange(0,groups), 0.8
                means = [pow.mean()for pow in pow_groups]
                stds = [stats.sem(pow) for pow in pow_groups]
                plt.bar(xticks,means,color='blue',yerr=stds)
                plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score'))
                plt.savefig(f_form_save.format(freq[2:],region,'EpiScore','z-score').replace('pdf','png'))
                plt.clf()
                plt.close()
            
pvals = np.array(pvals)[np.logical_not(np.isnan(pvals))]
## Determine the threshold p value for FDR correction
_, pval_bonf = bonferroni_correction(pvals, alpha=alpha)
print(np.round(pval_bonf,3))

_, pval_fdr = fdr_correction(pvals, alpha=alpha, method='indep')
print(np.round(pval_fdr,3))
print('H values', np.round(H_vals,3))

### Post-hoc statistics for kruskal Wallis H test

In [ ]:
from mne.stats import bonferroni_correction, fdr_correction
from dunn import kw_dunn

feat, corr, phase, win,th, alpha = 'pow', False, 'Encoding', 1,'0.01',0.05
groups = 5
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'feature/0_Power_'+phase+'_EpiPerf_By_Odor/')
npz_form = join(path_npz, '{}_odor_{}_bipo_sel_physFT_pow.npz')
path_classif = join(st.path, 'figure/0_Classif_Power_'+phase[0]+'_EpiPerf_LowHigh_1000perm/')
classif_npz = join(path_classif, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_form = join(path_classif, 'masks_stat/All_subjects_mask_stat_{}_minwin{}_th{}.npy') 
path2save = join(st.path, 'kruskal_pow_all_trials/'+phase+'/encoding_signif_elecs'+str(groups)+'/')
f_form_save = join(path2save, 'Kruskal_{}_{}_{}_{}.png')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

su_codes = [['S0','S1','S2','S3','S4','S5','S6'] if phase == 'Encoding' else ['S0','S1','S2','S3','S4','S5']]
freqs = ['2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method = ['s_Mail_RL']

if phase == 'Encoding':
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68}, #missing odor 15
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'MICP': {2:6,12:8,6:96,8:8,3:12,18:4,9:6,14:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}
else:
    odors_su = {'CHAF': {5:12,7:68,8:36,9:96,1:6,2:2,3:68,4:8},
            'LEFC': {1:4,2:0,3:6,4:12,14:96,15:2,16:4,17:68},
            'PIRJ': {4:36,9:2,1:4,18:32,6:34,5:4,7:68,15:4}, 
            'VACJ': {14:6,15:64,16:68,17:8,10:6,11:4,12:4,13:40},
            'SEMC': {10:2,11:6,12:6,13:6,5:8,7:4,8:8,9:10},
            'FERJ': {16:6,17:6,5:8,7:6,12:8,13:8,2:6,1:10}}

rois_list = {'Olf':['Amg','pPirT','Amg-PirT','Ins','OFC'],
            'MTL':['HC','PHG'],
             'Temporal':['FuG','ITG','MTG','STG']}
#'Frontal':['ACC','IFG','MFG','SFG'],
pvals = []
for i,freq in zip(range(2,7),freqs):
    for region in sorted(rois_list):
        subjects = np.load(classif_npz.format('All_subjects',freq, 'odor'))['su_codes']
        mask = np.load(masks_form.format(freq,str(win),th))
        mask = np.logical_not(mask) #inverse of visbrain 
        pow_odors, scores_odors, pow_raw = [], [], []
        for s,su in enumerate(sorted(odors_su)):
            mask_su = mask[np.where(subjects=='S'+str(s))]
            pow_su, score_su = [], []
            for roi in rois_list[region]:
                for odor in odors_su[su]:
                    arch_sig = np.load(npz_form.format(su,odor))
                    Mai_RL= arch_sig['Mai_RL'][mask_su]
                    #print(Mai_RL.shape, arch_sig['xpow'].shape)
                    power = np.mean(arch_sig['xpow'][i,:,27:47,:],axis=(1))[mask_su] #nfreq,nelecs,npts,ntrials
                    #power = arch_sig['xpow'][i,:,20,:][mask_su]
                    power_roi = power[np.where(Mai_RL==roi)]
                    power_roi = np.ndarray.flatten(power_roi)
                    score_roi = [odors_su[su][odor]]*power_roi.shape[0]
                    pow_su = np.append(pow_su,power_roi)
                    score_su = np.append(score_su,score_roi)
            pow_su_norm = (pow_su - pow_su.mean())/pow_su.std()
            score_su_norm = (score_su - score_su.mean())/score_su.std()
            pow_odors = np.append(pow_odors,pow_su_norm)
            scores_odors = np.append(scores_odors,score_su_norm)
            pow_raw = np.append(pow_raw,pow_su)
       #Split the data into 3 groups
        #print('before concat',scores_odors.shape, pow_odors.shape)
        if scores_odors.shape[0] != 0:
            steps = np.percentile(scores_odors,[0,20,40,60,80,100])
            #print(scores_odors,steps)
            pow_groups, scores_groups, pow_raw_groups = [], [], []
            for g in range(groups):
                pow_groups.append(pow_odors[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
                pow_raw_groups.append(pow_raw[np.where((steps[g]<=scores_odors)&(scores_odors<=(steps[g+1])))])
            #print([pow.shape for pow in pow_groups], [pow.shape for pow in pow_raw_groups])

            #Compute the statistics (Kruskal-Wallis)
            to_compare = [(0, 4),(1, 3),(2, 0),(2, 4)]
            #to_compare = [(0, 1),(1, 2),(2, 3),(3, 4)]
            H,p,Z,p_uncor,p_corr,rej = kw_dunn(pow_groups,to_compare=to_compare, alpha=0.05, method='fdr_bh')
            print('stats for',freq,region,p,np.round(p_uncor,3),rej)
            pvals = np.append(pvals,p)
            
_, pval_fdr = fdr_correction(pvals, alpha=alpha, method='indep')
print(np.round(pval_fdr,3))
